In [1]:
import torch
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_digits
from torch.utils.data import DataLoader,Dataset

In [2]:
X,y = load_digits(return_X_y=True)
X=X.flatten().reshape(-1,64)
X,y,X.shape,y.shape

(array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ..., 10.,  0.,  0.],
        [ 0.,  0.,  0., ..., 16.,  9.,  0.],
        ...,
        [ 0.,  0.,  1., ...,  6.,  0.,  0.],
        [ 0.,  0.,  2., ..., 12.,  0.,  0.],
        [ 0.,  0., 10., ..., 12.,  1.,  0.]]),
 array([0, 1, 2, ..., 8, 9, 8]),
 (1797, 64),
 (1797,))

In [3]:
s = StandardScaler()
X = s.fit_transform(X)
X

array([[ 0.        , -0.33501649, -0.04308102, ..., -1.14664746,
        -0.5056698 , -0.19600752],
       [ 0.        , -0.33501649, -1.09493684, ...,  0.54856067,
        -0.5056698 , -0.19600752],
       [ 0.        , -0.33501649, -1.09493684, ...,  1.56568555,
         1.6951369 , -0.19600752],
       ...,
       [ 0.        , -0.33501649, -0.88456568, ..., -0.12952258,
        -0.5056698 , -0.19600752],
       [ 0.        , -0.33501649, -0.67419451, ...,  0.8876023 ,
        -0.5056698 , -0.19600752],
       [ 0.        , -0.33501649,  1.00877481, ...,  0.8876023 ,
        -0.26113572, -0.19600752]])

In [4]:
X = torch.tensor(X,dtype=torch.float32)
y = torch.tensor(y,dtype=torch.long)

In [5]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [6]:
class CustomDataset(Dataset):
  def __init__(self,X,y):
    self.X = X
    self.y = y

  def __len__(self):
    return len(self.X)

  def __getitem__(self,idx):
    return self.X[idx],self.y[idx]

In [7]:
train_dataset = CustomDataset(X_train,y_train)
test_dataset = CustomDataset(X_test,y_test)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)


In [8]:
input_dim = 64
hidden_dim = 64
output_dim = 10

In [9]:
class SimpleNN(torch.nn.Module):
  def __init__(self):
    super(SimpleNN,self).__init__()
    self.fc1 = torch.nn.Linear(input_dim,hidden_dim)
    self.fc2 = torch.nn.Linear(hidden_dim,hidden_dim)
    self.fc3 = torch.nn.Linear(hidden_dim,output_dim)

  def forward(self,x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = torch.softmax(self.fc3(x),dim=1)
    return x



In [10]:
model = SimpleNN()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)

In [11]:
epochs = 100
for epoch in range(epochs):
  for i,(x,y) in enumerate(train_dataloader):
    # forward pass
    y_pred = model(x)
    loss = criterion(y_pred,y)
    optimizer.zero_grad()

    # backward pass
    loss.backward()
    optimizer.step()
  print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

Epoch 1/100, Loss: 2.3011484146118164
Epoch 2/100, Loss: 2.288142204284668
Epoch 3/100, Loss: 2.2500391006469727
Epoch 4/100, Loss: 2.225463390350342
Epoch 5/100, Loss: 2.097933292388916
Epoch 6/100, Loss: 2.081578254699707
Epoch 7/100, Loss: 1.7957916259765625
Epoch 8/100, Loss: 1.9035580158233643
Epoch 9/100, Loss: 1.8224632740020752
Epoch 10/100, Loss: 1.9153473377227783
Epoch 11/100, Loss: 1.6915968656539917
Epoch 12/100, Loss: 1.75942862033844
Epoch 13/100, Loss: 1.5966464281082153
Epoch 14/100, Loss: 1.57568359375
Epoch 15/100, Loss: 1.6032344102859497
Epoch 16/100, Loss: 1.4945299625396729
Epoch 17/100, Loss: 1.5626307725906372
Epoch 18/100, Loss: 1.5925840139389038
Epoch 19/100, Loss: 1.6172091960906982
Epoch 20/100, Loss: 1.499048113822937
Epoch 21/100, Loss: 1.5829837322235107
Epoch 22/100, Loss: 1.564858078956604
Epoch 23/100, Loss: 1.4734679460525513
Epoch 24/100, Loss: 1.4846292734146118
Epoch 25/100, Loss: 1.4769288301467896
Epoch 26/100, Loss: 1.514197587966919
Epoch 27/

In [12]:
correct =0

for i,(x,y) in enumerate(test_dataloader):
  y_pred = model(x)
  correct+=(y_pred.argmax(dim=1)==y).sum().item()
print(f"Accuracy: {correct/len(test_dataset)}")



Accuracy: 0.9777777777777777
